In [30]:
from base import np
import pandas as pd

import importlib
import threeD
importlib.reload(threeD)
from threeD import FluxMap3D

# Export figures to Folder
figures_path = "../Figures/FluxMap3D/"

In [31]:
def shell_generator(r0, l):
    'Define the inner step function that takes r as parameter'
    def c_shell(r):
        'Determine the condition based on r0 and l'
        cond = (r >= r0) & (r <= r0 + l)
        c = 3/(4*np.pi) * 1/(l**3 + 3 * l**2 * r0 + 3 * l * r0**2)
        # Return c where cond == True, else 0
        return np.where(cond , c , 0)
    return c_shell

#### Extract Real Numbers from Excel

In [32]:
# Load the Excel file
bionums_df = pd.read_excel('Real_Numbers.xlsx')

# Real Diatom Radii (w std) 
bionums_df['R_d (μm)'] = bionums_df['L_d (μm)'].values/2
bionums_df['σR_d'] = bionums_df['σL_d'].values/2

# Real Bacteria Radii (w std)
bionums_df['R_b (μm)'] = bionums_df['L_b (μm)'].values/2
bionums_df['σR_b'] = bionums_df['σL_b'].values/2

bionums_df

,Diatom,Diatom Length,L_d (μm),σL_d,Bacteria,Bacteria Length,L_b (μm),σL_b,Interaction Source,Diatom Length Source,Bacterial Length Source,R_d (μm),σR_d,R_b (μm),σR_b
0,Asterionellopsis glacialis,30 – 150 μm in length,90,60,Alteromonas macleodii,1–2 µm long,1.50,0.50,Residence time of a Brownian particle in a che...,EOS - Phytoplankton Encyclopedia Project,Microbe Profile: Alteromonas macleodii | Micro...,45.0,30.0,0.750,0.250
1,Thalassiosira weissflogii,4 – 32 µm in diameter,18,14,Marinobacter adhaerens,1.7-2.4 µm in length,2.05,0.35,Chemotaxis of Marinobacter adhaerens and its i...,Species Profile - Thalassiosira weissflogii,Marinobacter adhaerens HP15 | BacDive,9.0,7.0,1.025,0.175


#### Set the System's Parameters as a function of the Diatom Radius

In [33]:
syst_params_func = lambda R_dtm : {
    # Space Boundaries
    'R_dtm' : R_dtm, 'R_inf' : 1000*R_dtm,
    # Max Bacterial Boundary
    'L' : 3*R_dtm,
    'alpha': None,
    # Number of points
    'nr': 100_000, # Weird triangles appear if it's too low
    'n_rho': 250,'n_lambda': 250
}
SOLVE_SYSTEM = True

### 1. *Asterionellopsis glacialis* diatom & *Alteromonas macleodii* bacteria

In [34]:
# Extract the first row of the DataFrame
bionums = bionums_df.iloc[0]
R_d = bionums['R_d (μm)']
R_b = bionums['R_b (μm)']
R_bmax = bionums['R_b (μm)'] + bionums['σR_b']
label_d = bionums['Diatom']
label_b = bionums['Bacteria']

In [ ]:
syst_params = syst_params_func(int(R_d))
syst_params['alpha'] = 1e2

filename = f'c_shell(r)_{label_d.replace(" ", "_")}'

FM = FluxMap3D(syst_params, shell_generator)
if True:
    FM.solve(n_jobs=-1)
    FM.save_data(filename)
else:
    FM.load_data(filename)
FM.plot(flux_range=False, set_extremes=True, set_transition=True)

print(f'Diatom: {label_d}')

FM.ax.scatter(
    FM.x_values[4], FM.y_values[4], color='red', s=R_b**2 * 100,
    label=f'{label_b} bacterium\n(R = {R_b})'
    )

FM.ax.scatter(
    FM.x_values[4], FM.y_values[4], color= 'red', s=R_bmax**2 * 100,
    alpha=0.3
    )
FM.ax.legend()

print(f'Diatom Radius: {R_d}')
print(f'Bacterium Radius: {R_b} ± {bionums["σR_b"]}')

KeyError: "Unable to open object (object 'c_shell(r)_Asterionellopsis_glacialis' doesn't exist)"

#### Now, $R_d = 20$ μm

In [25]:
R_d = 20 # bionums['R_d (μm)']

In [ ]:
syst_params = syst_params_func(int(R_d))
syst_params['alpha'] = 1e2

FM = FluxMap3D(syst_params, shell_generator)
FM.solve(n_jobs=-1)
FM.title += f'\n for {label_d}'
FM.plot()
FM.ax.scatter(
    FM.x_values[4], FM.y_values[4], color='red', s=R_b**2 * 100,
    label=f'{label_b} bacterium\n(R = {R_b})'
    )
FM.ax.scatter(
    FM.x_values[4], FM.y_values[4], color= 'red', s=R_bmax**2 * 100,
    alpha=0.3
    )
FM.ax.legend()

print(f'Diatom Radius: {R_d}')
print(f'Bacterium Radius: {R_b} ± {bionums["σR_b"]}')

### *Thalassiosira weissflogii* diatom & *Marinobacter adhaerens* bacteria

In [26]:
bionums = bionums_df.iloc[1]
R_d = bionums['R_d (μm)']
R_b = bionums['R_b (μm)']
R_bmax = bionums['R_b (μm)'] + bionums['σR_b']
label_d = bionums['Diatom']
label_b = bionums['Bacteria']

In [ ]:
syst_params = syst_params_func(int(R_d))
syst_params['alpha'] = 1e2
filename = f'c_shell(r)_{label_d.replace(" ", "_")}'

FM = FluxMap3D(syst_params, shell_generator)
if SOLVE_SYSTEM:
    FM.solve(n_jobs=-1)
    FM.save_data(filename)
else:
    FM.load_data(filename)
FM.plot(flux_range=False, set_extremes=True, set_transition=True)
print(f'Diatom: {label_d}')

FM.ax.scatter(
    FM.x_values[4], FM.y_values[4], color='red', s=R_b**2 * 100,
    label=f'{label_b} bacterium\n(R = {R_b})'
    )
FM.ax.scatter(
    FM.x_values[4], FM.y_values[4], color= 'red', s=R_bmax**2 * 100,
    alpha=0.3
    )
FM.ax.legend();

print(f'Diatom Radius: {R_d}')
print(f'Bacterium Radius: {R_b} ± {bionums["σR_b"]}')